**[Intermediate Machine Learning Home Page](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


Now it's your turn to test your new knowledge of **missing values** handling. You'll probably find it makes a big difference.

# Setup

The questions will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
from os import listdir
from os.path import isfile, join

working_path = "c:/code/python/kaggle/House Price/"

In this exercise, you will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv(working_path + 'input/train.csv', index_col='Id')
X_test_full = pd.read_csv(working_path + 'input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
# X = X_full.select_dtypes(exclude=['object'])
# X_test = X_test_full.select_dtypes(exclude=['object'])
X = X_full
X_test = X_test_full

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Use the next code cell to print the first five rows of the data.

In [3]:
# Shape of training data (num_rows, num_columns)
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


# Preliminary investigation

In [4]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 79)
LotFrontage      212
Alley           1097
MasVnrType         6
MasVnrArea         6
BsmtQual          28
BsmtCond          28
BsmtExposure      28
BsmtFinType1      28
BsmtFinType2      29
Electrical         1
FireplaceQu      551
GarageType        58
GarageYrBlt       58
GarageFinish      58
GarageQual        58
GarageCond        58
PoolQC          1164
Fence            954
MiscFeature     1119
dtype: int64


# Prepare the data

## Step 1a - Label encoding - MANUAL

In [5]:
def label_encoding(df):
    # Label Encoding // CATEGORIES TO NUMBER
    df.Alley.fillna(value=0, inplace=True)
    df.BsmtQual.fillna(value=0, inplace=True)
    df.BsmtCond.fillna(value=0, inplace=True)
    df.BsmtExposure.fillna(value=0, inplace=True)
    df.BsmtFinType1.fillna(value=0, inplace=True)
    df.BsmtFinType2.fillna(value=0, inplace=True)
    df.FireplaceQu.fillna(value=0, inplace=True)
    df.GarageFinish.fillna(value=0, inplace=True)
    df.GarageQual.fillna(value=0, inplace=True)
    df.PoolQC.fillna(value=0, inplace=True)
    df.Fence.fillna(value=0, inplace=True)
    
    cleanup_nums = {"Alley":        {"Grvl": 1, "Pave": 2},
                    "BsmtQual":     {"Ex":5,"Gd": 4,"TA":3,"Fa": 2,"Po":1},
                    "BsmtCond":     {"Ex":5,"Gd": 4,"TA":3,"Fa": 2,"Po":1},
                    "BsmtExposure": {"Gd": 4, "Av": 3, "Mn": 2, "No": 1},
                    "BsmtFinType1": {"GLQ": 6,"ALQ": 5,"BLQ": 4,"Rec": 3,"LwQ": 2,"Unf": 1},
                    "BsmtFinType2": {"GLQ": 6,"ALQ": 5,"BLQ": 4,"Rec": 3,"LwQ": 2,"Unf": 1},
                    "FireplaceQu":  {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1},
                    "GarageFinish": {"Fin":3,"RFn":2,"Unf":1},
                    "GarageQual":   {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1 },
                    "PoolQC":       {"Ex":4,"Gd":3,"TA":2,"Fa":1},
                    "Fence":        {"GdPrv":4, "MnPrv":3, "GdWo":2, "MnWw":1}
                   }

    df.replace(cleanup_nums, inplace=True)
    return df

In [6]:
# Label Encoding // CATEGORIES TO NUMBER
X_train = label_encoding(X_train.copy())
X_valid = label_encoding(X_valid.copy())

## Step 1b - Label encoding - AUTO

In [7]:
from sklearn.preprocessing import LabelEncoder

def label_encoding_auto(df):
    #object_cols = ['LotConfig', 'Neighborhood', 'BldgType', 'CentralAir', 'SaleType']
    object_cols = ['LotConfig', 'Neighborhood', 'BldgType', 'CentralAir']

    # Make copy to avoid changing original data 
    label_df = df.copy()

    # Apply label encoder to each column with categorical data
    label_encoder = LabelEncoder()
    for col in object_cols:
        print(col)
        label_df[col] = label_encoder.fit_transform(df[col])

    return label_df

# Label Encoding // AUTO
X_train = label_encoding_auto(X_train.copy())
X_valid = label_encoding_auto(X_valid.copy())

LotConfig
Neighborhood
BldgType
CentralAir
LotConfig
Neighborhood
BldgType
CentralAir


## Step 2 - Drop columns with missing values (will not be treated)

In [8]:
# Drop columns in training and validation data
cols_to_drop = ['GarageYrBlt', 'GarageCond','Fence', 'MiscFeature','MiscVal','SaleType']
# 'MiscFeature','MiscVal', 'SaleType' foram excluídas por outros motivos

X_train.drop(cols_to_drop, axis=1, inplace=True)
X_valid.drop(cols_to_drop, axis=1, inplace=True)

## Step 3 - One Hot Encoding

In [9]:
def one_hot_encoding(df):
    # Electrical
    index = 6
    columns = ['Electrical_SBrkr', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_Null']
    df_electrical = pd.DataFrame(columns=columns)
    df_electrical['Electrical_SBrkr'] = df.Electrical == 'SBrkr'
    df_electrical['Electrical_FuseA'] = df.Electrical == 'FuseA'
    df_electrical['Electrical_FuseF'] = df.Electrical == 'FuseF'
    df_electrical['Electrical_FuseP'] = df.Electrical == 'FuseP'
    df_electrical['Electrical_Mix'] = df.Electrical == 'Mix'
    df_electrical['Electrical_Null'] = df.Electrical.isnull()

    # MasVnrType
    index = 4
    columns = ['MasVnrType_BrkFace', 'MasVnrType_Stone', 'MasVnrType_BrkCmn', 'MasVnrType_Null']
    df_masVnrType = pd.DataFrame(columns=columns)
    df_masVnrType['MasVnrType_BrkFace'] = df.MasVnrType == 'BrkFace'
    df_masVnrType['MasVnrType_Stone'] = df.MasVnrType == 'Stone'
    df_masVnrType['MasVnrType_BrkCmn'] = df.MasVnrType == 'BrkCmn'
    df_masVnrType['MasVnrType_Null'] = df.MasVnrType.isnull()

    # GarageType
    index = 7
    columns = ['GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 
               'GarageType_Detchd', 'GarageType_Null']
    df_garageType = pd.DataFrame(columns=columns)
    df_garageType['GarageType_2Types'] = df.GarageType == '2Types'
    df_garageType['GarageType_Attchd'] = df.GarageType == 'Attchd'
    df_garageType['GarageType_Basment'] = df.GarageType == 'Basment'
    df_garageType['GarageType_BuiltIn'] = df.GarageType == 'BuiltIn'
    df_garageType['GarageType_CarPort'] = df.GarageType == 'CarPort'
    df_garageType['GarageType_Detchd'] = df.GarageType == 'Detchd'
    df_garageType['GarageType_Null'] = df.GarageType.isnull()

    ## MiscFeature
    #index = 5
    #columns = ['MiscFeature_Elev', 'MiscFeature_Gar2', 'MiscFeature_Shed', 'MiscFeature_TenC', 'MiscFeature_Null']
    #df_miscFeature = pd.DataFrame(columns=columns)
    #df_miscFeature['MiscFeature_Elev'] = df.MiscFeature == 'Elev'
    #df_miscFeature['MiscFeature_Gar2'] = df.MiscFeature == 'Gar2'
    #df_miscFeature['MiscFeature_Shed'] = df.MiscFeature == 'Shed'
    #df_miscFeature['MiscFeature_TenC'] = df.MiscFeature == 'TenC'
    #df_miscFeature['MiscFeature_Null'] = df.MiscFeature.isnull()

    #OH_df = pd.concat([df_electrical, df_masVnrType, df_garageType, df_miscFeature], axis=1)
    OH_df = pd.concat([df_electrical, df_masVnrType, df_garageType], axis=1)
    return OH_df

In [10]:
OH_X_train = one_hot_encoding(X_train)
OH_X_valid = one_hot_encoding(X_valid)

#cols_to_drop = ['Electrical', 'MasVnrType', 'GarageType', 'MiscFeature']
cols_to_drop = ['Electrical', 'MasVnrType', 'GarageType']

# Remove categorical columns
droped_X_train = X_train.drop(cols_to_drop, axis=1)
droped_X_valid = X_valid.drop(cols_to_drop, axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([droped_X_train, OH_X_train], axis=1)
X_valid = pd.concat([droped_X_valid, OH_X_valid], axis=1)


# Drop text columns

In [11]:
# To keep things simple, we'll use only numerical predictors
X_train = X_train.select_dtypes(exclude=['object'])
X_valid = X_valid.select_dtypes(exclude=['object'])

# Checking for missing values again

In [12]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 65)
LotFrontage    212
MasVnrArea       6
dtype: int64


# Imputation

In [13]:
from sklearn.impute import SimpleImputer

# Imputation
imputer = SimpleImputer() 
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# FImputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

X_train = imputed_X_train
X_valid = imputed_X_valid

# Select a model

### RandonForestRegressor

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

In [15]:
print("MAE (Drop columns with missing values):")
print(mean_absolute_error(y_valid, preds))

MAE (Drop columns with missing values):
17145.169965753426


### XGBRegressor

In [1]:
from xgboost import XGBRegressor

# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=0)

# Fit the model
my_model_2.fit(X_train, y_train, 
             early_stopping_rounds=100,
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

ModuleNotFoundError: No module named 'xgboost'

# SUBMISSION - Generate test predictions

In [16]:
# Label Encoding // CATEGORIES TO NUMBER
X_test = label_encoding(X_test.copy())

# Label Encoding // AUTO
X_test = label_encoding_auto(X_test.copy())


## X_test  ON HOT ENCODER ##
OH_X_test = one_hot_encoding(X_test)

cols_to_drop = ['Electrical', 'MasVnrType', 'GarageType']

# Remove categorical columns
droped_X_test = X_test.drop(cols_to_drop, axis=1)
# Add one-hot encoded columns to numerical features
X_test = pd.concat([droped_X_test, OH_X_test], axis=1)


# Drop columns we don't want to use
cols_to_drop = ['GarageYrBlt', 'GarageCond','Fence','MiscFeature','MiscVal','SaleType']
X_test.drop(cols_to_drop, axis=1, inplace=True)


# Drop text columns
X_test = X_test.select_dtypes(exclude=['object'])


# Shape of training data (num_rows, num_columns)
print(X_test.shape)

LotConfig
Neighborhood
BldgType
CentralAir
(1459, 65)


In [17]:
#from sklearn.impute import SimpleImputer

# Imputation
#imputer = SimpleImputer() 
imputed_X_test = pd.DataFrame(imputer.fit_transform(X_test))

# FImputation removed column names; put them back
imputed_X_test.columns = X_test.columns
imputed_X_test.index = X_test.index
X_test = imputed_X_test

In [18]:
# Number of missing values in each column of training data
missing_val_count_by_column = (X_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


# Generate submission

In [19]:
#model = RandomForestRegressor(n_estimators=100, random_state=0)
#model.fit(X_train, y_train)
#preds = model.predict(X_valid)
#print(mean_absolute_error(y_valid, preds))

preds_test = model.predict(X_test)

In [20]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv(working_path + 'output/submission.csv', index=False)

# Step 5: Submit your results

Once you have successfully completed Step 4, you're ready to submit your results to the leaderboard!  (_You also learned how to do this in the previous exercise.  If you need a reminder of how to do this, please use the instructions below._)  

First, you'll need to join the competition if you haven't already.  So open a new window by clicking on [this link](https://www.kaggle.com/c/home-data-for-ml-course).  Then click on the **Join Competition** button.

![join competition image](https://i.imgur.com/wLmFtH3.png)

Next, follow the instructions below:
1. Begin by clicking on the blue **Save Version** button in the top right corner of the window.  This will generate a pop-up window.  
2. Ensure that the **Save and Run All** option is selected, and then click on the blue **Save** button.
3. This generates a window in the bottom left corner of the notebook.  After it has finished running, click on the number to the right of the **Save Version** button.  This pulls up a list of versions on the right of the screen.  Click on the ellipsis **(...)** to the right of the most recent version, and select **Open in Viewer**.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the **Output** tab on the right of the screen.  Then, click on the blue **Submit** button to submit your results to the leaderboard.

You have now successfully submitted to the competition!

If you want to keep working to improve your performance, select the blue **Edit** button in the top right of the screen. Then you can change your code and repeat the process. There's a lot of room to improve, and you will climb up the leaderboard as you work.


# Keep going

Move on to learn what **[categorical variables](https://www.kaggle.com/alexisbcook/categorical-variables)** are, along with how to incorporate them into your machine learning models.  Categorical variables are very common in real-world data, but you'll get an error if you try to plug them into your models without processing them first!

---
**[Intermediate Machine Learning Home Page](https://www.kaggle.com/learn/intermediate-machine-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161289) to chat with other Learners.*